In [3]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum

# Load the data
bakery_data = pd.read_csv('bakery.csv')

# Correcting the 'Daypart' to have consistent capitalization
bakery_data['Daypart'] = bakery_data['Daypart'].str.capitalize()

# Create a multi-index for the items and dayparts
item_daypart = pd.MultiIndex.from_frame(bakery_data[['Item', 'Daypart']])

# Initialize the model
model = Model('BakeryOptimization')

# Decision variables: 
X = model.addVars(item_daypart, vtype=GRB.INTEGER, name="X")
produce_item = model.addVars(item_daypart, vtype=GRB.BINARY, name="Produce")

# Parameters
profits = bakery_data.set_index(['Item', 'Daypart'])['Profit'].to_dict()
carbon_footprints = bakery_data.set_index(['Item', 'Daypart'])['Carbon Footprint'].to_dict()
average_sales = bakery_data.set_index(['Item', 'Daypart'])['Average Sale'].to_dict()
customer_ratings = bakery_data.set_index(['Item', 'Daypart'])['Customer Rating'].to_dict()
average_daily_profit = 500

# Objective: Minimize the total daily carbon footprint
model.setObjective(quicksum(carbon_footprints[i] * X[i] for i in item_daypart), GRB.MINIMIZE)

# Constraints

# Constraint 1: The total profit must be at least the average daily profit
model.addConstr(quicksum(profits[i] * X[i] for i in item_daypart) >= average_daily_profit, 'MinProfit')

# Constraint 2: Cap the total production to x items per day
model.addConstr(quicksum(X[i] for i in item_daypart) >= 130, 'TotalProductionCap')

# Constraint 3: Ensure at least 5 different items are produced in each Morning and Afternoon, and at least 1 item for each Evening and Night
min_items_per_daypart = {'Morning': 5, 'Afternoon': 5, 'Evening': 1, 'Night': 1}  # Dictionary to hold minimum items per daypart

for daypart in ['Morning', 'Afternoon', 'Evening', 'Night']:
    items_in_daypart = bakery_data[bakery_data['Daypart'] == daypart]['Item'].unique()
    min_items = min_items_per_daypart[daypart]  # Get the minimum number of items for the current daypart
    # Add a constraint that the sum of the binary variables for each daypart should be at least the minimum number
    model.addConstr(quicksum(produce_item[item, daypart] for item in items_in_daypart) >= min_items, f'MinDiffItems_{daypart}')

# Constraint 4: If an item is produced in a specific daypart, its production quantity should be at most 15
for item, daypart in item_daypart:
    max_prod = min(15, average_sales[item, daypart] + 1)  # The maximum of 15 or average sales + 1
    model.addConstr(X[item, daypart] <= max_prod, name=f"max_production_{item}_{daypart}")


# Constraint 5: Do not produce items with average sales of 2 or less
for item, daypart in item_daypart:
    if average_sales[item, daypart] <= 2:
        model.addConstr(X[item, daypart] == 0, name=f"no_production_low_sales_{item}_{daypart}")

# Constraint 6: If an item is produced, produce greater than its average sale - 1
for item, daypart in item_daypart:
    model.addConstr(X[item, daypart] >= (produce_item[item, daypart] * (average_sales[item, daypart] - 1)), name=f"produce_more_than_average_{item}_{daypart}")

# Constraint 7: Produce items with average customer rating for the entire day greater than 4
total_rating = quicksum(customer_ratings[i] * X[i] for i in item_daypart)
total_production = quicksum(X[i] for i in item_daypart)
model.addConstr(total_rating >= 4.0 * total_production, 'MinAverageRating')   

# Optimize the model
model.optimize()

# Prepare to output the results
if model.status == GRB.OPTIMAL or model.status == GRB.SUBOPTIMAL:
    # Retrieve and print the production plan
    production_plan = {v.varName: v.x for v in model.getVars() if 'X' in v.varName and v.x > 0}
    for item_daypart, quantity in production_plan.items():
        print(f"{item_daypart}: {quantity}")
    objective_value = model.ObjVal
    print(f"Objective Value (Minimized Carbon Footprint): {objective_value}")


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 554 rows, 390 columns and 1410 nonzeros
Model fingerprint: 0x55e569e4
Variable types: 0 continuous, 390 integer (195 binary)
Coefficient statistics:
  Matrix range     [5e-02, 3e+01]
  Objective range  [1e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Presolve removed 551 rows and 365 columns
Presolve time: 0.02s
Presolved: 3 rows, 25 columns, 70 nonzeros
Variable types: 0 continuous, 25 integer (0 binary)
Found heuristic solution: objective 48.4000000

Root relaxation: objective 4.275603e+01, 4 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   42.75603    0    3   48.40000   42.75603  